# EXcercise Chapter 8

9. Load the MNIST dataset (introduced in Chapter 3) and split it into a training set and a test set (take the first 60,000 instances for training, and the remaining 10,000 for testing). Train a Random Forest classifier on the dataset and time how long it takes, then evaluate the resulting model on the test set. Next, use PCA to reduce the dataset’s dimensionality, with an explained variance ratio of 95%.Train a new Random Forest classifier on the reduced dataset and see how long it takes. Was training much faster? Next, evaluate the classifier on the test set. How does it compare to the previous classifier? 

---

10. Use t-SNE to reduce the MNIST dataset down to two dimensions and plot the result using Matplotlib. You can use a scatterplot using different colors to rep‐resent each image’s target class. Alternatively, you can replace each dot in the scatterplot with the corresponding instance’s class (a digit from 0 to 9), or even plot scaled-down versions of the digit images themselves (if you plot all digits, the visualization will be too cluttered, so you should either draw a random sam‐ple or plot an instance only if no other instance has already been plotted at a close distance). You should get a nice visualization with well-separated clusters of digits. Try using other dimensionality reduction algorithms such as PCA, LLE, or MDS and compare the resulting visualizations.

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

In [5]:
X = pd.read_feather('../data/mnist_784_X.feather')
y = pd.read_feather('../data/mnist_784_y.feather')
X = X.to_numpy()
y = y.to_numpy()
y = y.ravel()

X_train = X[:60000]
y_train = y[:60000]
X_test = X[60000:]
y_test = y[60000:]

*Exercise: Train a Random Forest classifier on the dataset and time how long it takes, then evaluate the resulting model on the test set.*

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA 
import time

In [15]:
rnd_clf = RandomForestClassifier(n_estimators=100,random_state=42)

In [16]:
t_0 = time.time()
rnd_clf.fit(X_train, y_train)
t_1 = time.time()
t_d = round((t_1 - t_0),2)
print(f"RandomForest training took {t_d} s.")

RandomForest training took 26.8 s.


In [17]:
y_pred = rnd_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9705

- Now use PCA with 95% explained variance
- Then Train a new Random Forest classifier on the reduced dataset and see how long it takes. 
    - Was training much faster?

In [19]:
pca = PCA(n_components=0.95)
X_train_reduced =pca.fit_transform(X_train)

In [24]:
rnd_clf_2 = RandomForestClassifier(n_estimators =100,random_state =42)

In [25]:
t_0 = time.time() # Run Forest! Run!
rnd_clf_2.fit(X_train_reduced, y_train)
t_1 = time.time()
t_d = round((t_1 - t_0),2)
print(f"With the reduced dataset,RandomForest training took {t_d} s.")

With the reduced dataset,RandomForest training took 74.38 s.


In [29]:
X_test_reduced = pca.transform(X_test)
y_pred = rnd_clf_2.predict(X_test_reduced)
accuracy_score(y_test, y_pred)

0.9481

- Dimensionality reduction does not always lead to faster training time: it depends on the dataset, the model and the training algorithm. See figure 8-6 (the `manifold_decision_boundary_plot*` plots above). If you try a softmax classifier instead of a random forest classifier, you will find that training time is reduced by a factor of 3 when using PCA

---

- It is common for performance to drop slightly when reducing dimensionality, because we do lose some useful signal in the process. However, the performance drop is rather severe in this case. So PCA really did not help: it slowed down training and reduced performance. :(

Let's see if it helps when using softmax regression:

In [31]:
from sklearn.linear_model import LogisticRegression

In [32]:
log_clf = LogisticRegression(multi_class="multinomial",solver='lbfgs',random_state=42)

In [33]:
t_0 = time.time()
log_clf.fit(X_train, y_train)
t_1 = time.time()
t_d = round((t_1 - t_0),2)
print(f"Logistic Regression training took {t_d} s.")

Logistic Regression training took 11.96 s.


/opt/homebrew/Caskroom/miniconda/base/envs/p39/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
